### Netflix Recommendation System
-----

Using the cosine similiraty and a Netflix Dataset I create a recommendation system that recommedns you 15 movies that are alike the oen you like, in this case "Austin Power: Goldmember"

------


In [95]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity


In [4]:
df = pd.read_csv('../Recommendation System/netflixData.csv')

In [6]:
df.head()

,Show Id,Title,Description,Director,Genres,Cast,Production Country,Release Date,Rating,Duration,Imdb Score,Content Type,Date Added
0,cc1b6ed9-cf9e-4057-8303-34577fb54477,(Un)Well,This docuseries takes a deep dive into the luc...,NaN,Reality TV,NaN,United States,2020.0,TV-MA,1 Season,6.6/10,TV Show,NaN
1,e2ef4e91-fb25-42ab-b485-be8e3b23dedb,#Alive,"As a grisly virus rampages a city, a lone man ...",Cho Il,"Horror Movies, International Movies, Thrillers","Yoo Ah-in, Park Shin-hye",South Korea,2020.0,TV-MA,99 min,6.2/10,Movie,"September 8, 2020"
2,b01b73b7-81f6-47a7-86d8-acb63080d525,#AnneFrank - Parallel Stories,"Through her diary, Anne Frank's story is retol...","Sabina Fedeli, Anna Migotto","Documentaries, International Movies","Helen Mirren, Gengher Gatti",Italy,2019.0,TV-14,95 min,6.4/10,Movie,"July 1, 2020"
3,b6611af0-f53c-4a08-9ffa-9716dc57eb9c,#blackAF,Kenya Barris and his family navigate relations...,NaN,TV Comedies,"Kenya Barris, Rashida Jones, Iman Benson, Genn...",United States,2020.0,TV-MA,1 Season,6.6/10,TV Show,NaN
4,7f2d4170-bab8-4d75-adc2-197f7124c070,#cats_the_mewvie,This pawesome documentary explores how our fel...,Michael Margolis,"Documentaries, International Movies",NaN,Canada,2020.0,TV-14,90 min,5.1/10,Movie,"February 5, 2020"


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5967 entries, 0 to 5966
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Show Id             5967 non-null   object 
 1   Title               5967 non-null   object 
 2   Description         5967 non-null   object 
 3   Director            3903 non-null   object 
 4   Genres              5967 non-null   object 
 5   Cast                5437 non-null   object 
 6   Production Country  5408 non-null   object 
 7   Release Date        5964 non-null   float64
 8   Rating              5963 non-null   object 
 9   Duration            5964 non-null   object 
 10  Imdb Score          5359 non-null   object 
 11  Content Type        5967 non-null   object 
 12  Date Added          4632 non-null   object 
dtypes: float64(1), object(12)
memory usage: 606.1+ KB


In [15]:
df.isnull().sum()

Show Id                  0
Title                    0
Description              0
Director              2064
Genres                   0
Cast                   530
Production Country     559
Release Date             3
Rating                   4
Duration                 3
Imdb Score             608
Content Type             0
Date Added            1335
dtype: int64

In [16]:
df.columns

Index(['Show Id', 'Title', 'Description', 'Director', 'Genres', 'Cast',
       'Production Country', 'Release Date', 'Rating', 'Duration',
       'Imdb Score', 'Content Type', 'Date Added'],
      dtype='object')

In [96]:
df1 = df[['Title', 'Description', 'Content Type', 'Genres']]

In [97]:
df1

,Title,Description,Content Type,Genres
0,(Un)Well,This docuseries takes a deep dive into the luc...,TV Show,Reality TV
1,#Alive,"As a grisly virus rampages a city, a lone man ...",Movie,"Horror Movies, International Movies, Thrillers"
2,#AnneFrank - Parallel Stories,"Through her diary, Anne Frank's story is retol...",Movie,"Documentaries, International Movies"
3,#blackAF,Kenya Barris and his family navigate relations...,TV Show,TV Comedies
4,#cats_the_mewvie,This pawesome documentary explores how our fel...,Movie,"Documentaries, International Movies"
...,...,...,...,...
5962,الف مبروك,"On his wedding day, an arrogant, greedy accoun...",Movie,"Comedies, Dramas, International Movies"
5963,دفعة القاهرة,A group of women leaves Kuwait to attend unive...,TV Show,"International TV Shows, TV Dramas"
5964,海的儿子,"Two brothers start a new life in Singapore, wh...",TV Show,"International TV Shows, TV Dramas"
5965,반드시 잡는다,After people in his town start turning up dead...,Movie,"Dramas, International Movies, Thrillers"


In [20]:
df1.isnull().sum()

Title           0
Description     0
Content Type    0
Genres          0
dtype: int64

-------

### Data Cleaning

Because the Title column have a lot of characaters I create a function to clean the column

-----

In [99]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stemmer = nltk.SnowballStemmer('english')
import string
stopword = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/francisco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [100]:
def clean(text):
    text = re.sub('\[.*?\]', '' , text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('https://\S+|www\.\S+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation),'', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    #With this line I'm only keeping the non-stopwords in my 'text'
    text = ' '.join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text = ' '.join(text)
    
    return text
    

In [101]:
df1['Title'] = df1['Title'].apply(clean)

/var/folders/7l/s74834vd66g986g2_n1pgmfw0000gn/T/ipykernel_60463/491710775.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Title'] = df1['Title'].apply(clean)


In [102]:
df1['Title'].sample(10)

160             a twelv year night
4585    terrac hous boy  girl citi
447                         audibl
2202                   ice fantasi
1849                      god call
1229                      deadwind
4814                the final tabl
4434                   strang voic
695                black is beltza
4849                  the girl gun
Name: Title, dtype: object

In [103]:
feature = df1['Genres'].tolist()

In [151]:
tfidf = text.TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(feature)
similarity = cosine_similarity(tfidf_matrix)

In [108]:
indices = pd.Series(df1.index, index= df1['Title']).drop_duplicates()

In [117]:
indices['unwel']

0

In [152]:
def recommendation_system(title, similarity=similarity):
    index = indices[title]
    similarity_scores = list(enumerate(similiraty[index]))
    similarity_scores = sorted(similarity_scores, key= lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[0:15]
    movies_indices = [i[0] for i in similarity_scores]
    
    return df1['Title'].iloc[movies_indices]
    

In [154]:
df1['Title'].sample(20)

455            austin power goldmemb
2122       hot gimmick girl meet boy
822                          bromanc
4661               the birth reborn 
867                      call midwif
3557                  ordinari peopl
5727                           wazir
4909                       the inlaw
1349                             don
3089    mike epp dont take it person
2825                     love nagger
2512                   kengan ashura
1456                       el cartel
3076                    midnight sun
5566                     ultraviolet
3156                         mom war
1199                      dash  lili
5623              up among  the star
2067                      hire woman
345                an egyptian stori
Name: Title, dtype: object

In [155]:
recommendation_system('austin power goldmemb')

320     america the motion pictur
333                american ultra
455         austin power goldmemb
631              bever hill ninja
838                  bulletproof 
940                    char angel
1060                coffe  kareem
1095                      cop out
1410                     due date
1762                game over man
1920             gunpowd milkshak
4371           spenser confidenti
4407               starski  hutch
4786                   the doover
4875          the happytim murder
Name: Title, dtype: object